In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

import argparse
import sys
import warnings

import keras
from keras.preprocessing import image
from keras.utils import multi_gpu_model
import tensorflow as tf
from datetime import date

import losses
import models
from callbacks import RedirectModel, Evaluate
from models.retinanet import retinanet_bbox
from generators.csv_generator import CSVGenerator
from generators.voc_generator import PascalVocGenerator
from utils.anchors import make_shapes_callback
from utils.config import read_config_file, parse_anchor_parameters
from utils.model import freeze as freeze_model
from utils.transform import random_transform_generator
from utils.image import random_visual_effect_generator

from train import *

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [3]:
args = '--backbone resnet50 \
--lr 1e-5 \
--snapshot-path ./snapshots/pascal_resnet50 \
--batch-size 24 \
--epochs 100 \
--no-evaluation \
--image-min-side 512 \
--image-max-side 512 \
--loss piou_l1 \
--loss_weight 2 \
csv --annotations_path /datasets/dataset/coco2017/annotations/instances_train2017_obb_cat_dog.csv \
--classes_path /datasets/dataset/coco2017/annotations/classes_cat_dog.csv \
--base_dir_train /datasets/dataset/coco2017/train2017'

args = parse_args(args.split(' '))

{'dataset_type': 'csv', 'snapshot': None, 'imagenet_weights': True, 'weights': None, 'backbone': 'resnet50', 'batch_size': 24, 'gpu': None, 'num_gpus': 0, 'multi_gpu_force': False, 'epochs': 100, 'steps': 10000, 'lr': 1e-05, 'snapshot_path': './snapshots/pascal_resnet50', 'tensorboard_dir': 'logs/2021-05-04', 'snapshots': True, 'evaluation': False, 'freeze_backbone': False, 'random_transform': False, 'image_min_side': 512, 'image_max_side': 512, 'config': None, 'weighted_average': False, 'compute_val_loss': False, 'loss': 'piou_l1', 'loss_weight': 2.0, 'multiprocessing': False, 'workers': 1, 'max_queue_size': 10, 'annotations_path': '/datasets/dataset/coco2017/annotations/instances_train2017_obb_cat_dog.csv', 'classes_path': '/datasets/dataset/coco2017/annotations/classes_cat_dog.csv', 'base_dir_train': '/datasets/dataset/coco2017/train2017', 'val_annotations_path': None}


In [5]:
# create object that stores backbone information
backbone = models.backbone(args.backbone)

In [6]:
# create the generators
train_generator, validation_generator = create_generators(args, backbone.preprocess_image)

line 956: y2 (189.5) must be higher than y1 (189.5)
line 957: y2 (189.5) must be higher than y1 (189.5)
line 958: y2 (189.5) must be higher than y1 (189.5)
line 959: y2 (189.5) must be higher than y1 (189.5)


In [11]:
# create the model
if args.snapshot is not None:
    print('Loading model, this may take a second...')
    model = models.load_model(args.snapshot, backbone_name=args.backbone)
    training_model = model
    anchor_params = None
    if args.config and 'anchor_parameters' in args.config:
        anchor_params = parse_anchor_parameters(args.config)
    prediction_model = retinanet_bbox(model=model, anchor_params=anchor_params)
    # compile model
    training_model.compile(
        loss={
            'regression': losses.iou_loss(args.loss, args.loss_weight),
            'classification': losses.focal(),
            'centerness': losses.bce(),
        },
        optimizer=keras.optimizers.Adam(lr=1e-5)
        # optimizer=keras.optimizers.sgd(lr=1e-5, momentum=0.9, decay=1e-5, nesterov=True)
    )

else:
    weights = args.weights
    # default to imagenet if nothing else is specified
    if weights is None and args.imagenet_weights:
        weights = backbone.download_imagenet()

    print('Creating model, this may take a second...')
    model, training_model, prediction_model = create_models(
        backbone_retinanet=backbone.retinanet,
        num_classes=train_generator.num_classes(),
        weights=weights,
        num_gpus=args.num_gpus,
        freeze_backbone=args.freeze_backbone,
        lr=args.lr,
        config=args.config,
        args=args
    )

Creating model, this may take a second...


W0504 01:11:27.572842 140465654658880 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0504 01:11:27.579738 140465654658880 deprecation.py:323] From /workdir/msc/cell-detector/keras-fcos/losses.py:147: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


ValueError: Shapes must be equal rank, but are 0 and 1 for 'loss/regression_loss/Select' (op: 'Select') with input shapes: [?], [], [?].

In [7]:
parallel_model = multi_gpu_model(training_model, gpus=2)
parallel_model.compile(
    loss={
        'regression': losses.iou(),
        'classification': losses.focal(),
        'centerness': losses.bce(),
    },
    optimizer=keras.optimizers.Adam(lr=1e-4)
    # optimizer=keras.optimizers.sgd(lr=1e-5, momentum=0.9, decay=1e-5, nesterov=True)
)

# this lets the generator compute backbone layer shapes using the actual backbone model
if 'vgg' in args.backbone or 'densenet' in args.backbone:
    train_generator.compute_shapes = make_shapes_callback(model)
    if validation_generator:
        validation_generator.compute_shapes = train_generator.compute_shapes

W0504 01:05:15.659440 140465654658880 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0504 01:05:15.660485 140465654658880 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0504 01:05:15.662512 140465654658880 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0504 01:05:15.679048 140465654658880 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1919: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0504 01:05:15.681363 140465654

Creating model, this may take a second...


W0504 01:05:16.705312 140465654658880 deprecation_wrapper.py:119] From /workdir/msc/cell-detector/keras-fcos/utils_graph.py:108: The name tf.image.resize_images is deprecated. Please use tf.image.resize instead.



ValueError: slice index 4 of dimension 2 out of bounds. for 'boxes/strided_slice_8' (op: 'StridedSlice') with input shapes: [?,?,4], [3], [3], [3] and with computed input tensors: input[1] = <0 0 4>, input[2] = <0 0 5>, input[3] = <1 1 1>.

In [8]:
if not args.compute_val_loss:
    validation_generator = None

# start training
parallel_model.fit_generator(
    generator=train_generator,
    initial_epoch=0,
    steps_per_epoch=len(train_generator),
    epochs=args.epochs,
    verbose=1,
    callbacks=create_callbacks(
        model,
        training_model,
        prediction_model,
        validation_generator,
        args,
    ),
    validation_data=validation_generator
)

W0407 03:44:49.768165 139664585299776 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0407 03:44:49.768903 139664585299776 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/100
 15/586 [..............................] - ETA: 15:02 - loss: 1.3379 - regression_loss: 0.6512 - classification_loss: 0.1125 - centerness_loss: 0.5742

/workdir/msc/cell-detector/keras-fcos/generators/generator.py:258: UserWarning: Image with id 4901 (shape (375, 500, 3)) contains no valid boxes after transform
  image.shape,


164/586 [=======>......................] - ETA: 5:30 - loss: 0.9263 - regression_loss: 0.2673 - classification_loss: 0.0895 - centerness_loss: 0.5695

/workdir/msc/cell-detector/keras-fcos/generators/generator.py:258: UserWarning: Image with id 12579 (shape (333, 500, 3)) contains no valid boxes after transform
  image.shape,


...

Running network: 100% (2510 of 2510) |###| Elapsed Time: 0:01:04 Time:  0:01:04
Parsing annotations: 100% (2510 of 2510) || Elapsed Time: 0:00:00 Time:  0:00:00


155 instances of class aeroplane with average precision: 0.7882
177 instances of class bicycle with average precision: 0.6942
243 instances of class bird with average precision: 0.5876
150 instances of class boat with average precision: 0.4997
252 instances of class bottle with average precision: 0.3765
114 instances of class bus with average precision: 0.6936
625 instances of class car with average precision: 0.7079
190 instances of class cat with average precision: 0.8570
398 instances of class chair with average precision: 0.4285
123 instances of class cow with average precision: 0.6032
112 instances of class diningtable with average precision: 0.5840
257 instances of class dog with average precision: 0.7708
180 instances of class horse with average precision: 0.7615
172 instances of class motorbike with average precision: 0.7096
2332 instances of class person with average precision: 0.6886
266 instances of class pottedplant with average precision: 0.3230
127 instances of class shee

E0407 04:04:31.420890 139664585299776 ultratb.py:152] Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-e07dec684551>", line 18, in <module>
    validation_data=validation_generator
  File "/usr/local/lib/python3.6/dist-packages/keras/legacy/interfaces.py", line 91, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training.py", line 1418, in fit_generator
    initial_epoch=initial_epoch)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py", line 217, in fit_generator
    class_weight=class_weight)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training.py", line 1217, in train_on_batch
    outputs = self.train_function(ins)
  File "/usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py", line 2715, in __call__
    return self._call(inputs)
  File "/usr/loca

TypeError: object of type 'NoneType' has no len()

In [ ]:
training_model.save(os.path.join(args.snapshot_path, 'saved_model.h5'))

In [ ]:
os.path.join(args.snapshot_path, 'saved_model.h5')